# Problem 1 

![problem1](problem1.png)

In [1]:
from pyomo.environ import *

# Create a new model
model = ConcreteModel()

# Define the nodes in the network
nodes = ['DET', 'MIN', 'KAN', 'DEN', 'ALB', 'BOI', 'SAC','SPO','POR']

# Define the arcs in the network and their capacities
# Since the numbers on the map is in tons, we have to *2 so that it would be the number of engines that can be shipped.
arc_capacities = {('DET', 'MIN'): 80, ('KAN', 'MIN'): 40,
                  ('DET', 'DEN'): 80, ('KAN', 'ALB'): 40,
                  ('DET', 'KAN'): 40, ('DEN', 'BOI'): 40,
                  ('MIN', 'KAN'): 20, ('DEN', 'SAC'): 60,
                  ('MIN', 'SPO'): 60, ('DEN', 'ALB'): 20,
                  ('ALB', 'DEN'): 40, ('ALB', 'SAC'): 60,
                  ('BOI', 'POR'): 60, ('SAC', 'POR'): 100,
                  ('SPO', 'POR'): 80}

# Define the variable for the flow on each arc, bounded by the arc capacities
# upper and lower bound 
model.flow = Var(arc_capacities.keys(), domain=NonNegativeReals, bounds=lambda model, i, j: (0, arc_capacities[i, j]))

# Objective: Maximize the flow into LA
model.objective = Objective(expr=sum(model.flow[i, j] for (i, j) in arc_capacities.keys() if j == 'POR'), sense=maximize)

# Flow balance constraints: For each node except POR and DET, the inflow must equal the outflow
def flow_balance_rule(model, node):
    inflow = sum(model.flow[i, j] for (i, j) in arc_capacities.keys() if j == node)
    outflow = sum(model.flow[i, j] for (i, j) in arc_capacities.keys() if i == node)
    if node == 'DET':
        return outflow == sum(model.flow[node, j] for j in nodes if (node, j) in arc_capacities.keys())  # Source node: only outflow
    elif node == 'POR':
        return inflow == sum(model.flow[i, node] for i in nodes if (i, node) in arc_capacities.keys())  # Sink node: only inflow
    else:
        return inflow == outflow  # Intermediate nodes: inflow must equal outflow

model.flow_balance = Constraint(nodes, rule=flow_balance_rule)

# Display the model
model.pprint()

2 Set Declarations
    flow_balance_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    9 : {'DET', 'MIN', 'KAN', 'DEN', 'ALB', 'BOI', 'SAC', 'SPO', 'POR'}
    flow_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     2 :    Any :   15 : {('ALB', 'DEN'), ('ALB', 'SAC'), ('BOI', 'POR'), ('DEN', 'ALB'), ('DEN', 'BOI'), ('DEN', 'SAC'), ('DET', 'DEN'), ('DET', 'KAN'), ('DET', 'MIN'), ('KAN', 'ALB'), ('KAN', 'MIN'), ('MIN', 'KAN'), ('MIN', 'SPO'), ('SAC', 'POR'), ('SPO', 'POR')}

1 Var Declarations
    flow : Size=15, Index=flow_index
        Key            : Lower : Value : Upper : Fixed : Stale : Domain
        ('ALB', 'DEN') :     0 :  None :    40 : False :  True : NonNegativeReals
        ('ALB', 'SAC') :     0 :  None :    60 : False :  True : NonNegativeReals
        ('BOI', 'POR') :     0 :  None :    60 : False :  True : NonNegativeReals
        ('DEN',

In [2]:
from pyomo.environ import *
# [Your existing model definition here...]

# Solve the model using the Gurobi solver
solver = SolverFactory('gurobi')
result = solver.solve(model).write()

print("Objective Function Value: ", model.objective())

# Print the values of all decision variables
print("Decision Variable Values:")
for arc in model.flow:
    print(f"Flow on arc {arc}: {model.flow[arc].value}")

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x1
  Lower bound: 180.0
  Upper bound: 180.0
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 16
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 16
  Number of nonzeros: 24
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject

![problem2](problem2.png)

In [ ]:
from pyomo.environ import *

# Create a new model
model = ConcreteModel()

# Define the nodes in the network
nodes = ['DET', 'MIN', 'KAN', 'DEN', 'ALB', 'BOI', 'SAC','SPO','POR']

# Define the arcs in the network and their capacities
# Since the numbers on the map is in tons, we have to *2 so that it would be the number of engines that can be shipped.
arc_capacities = {('DET', 'MIN'): 80, ('KAN', 'MIN'): 40,
                  ('DET', 'DEN'): 80, ('KAN', 'ALB'): 40,
                  ('DET', 'KAN'): 40, ('DEN', 'BOI'): 40,
                  ('MIN', 'KAN'): 20, ('DEN', 'SAC'): 60,
                  ('MIN', 'SPO'): 60, ('DEN', 'ALB'): 20,
                  ('ALB', 'DEN'): 40, ('ALB', 'SAC'): 60,
                  ('BOI', 'POR'): 60, ('SAC', 'POR'): 100,
                  ('SPO', 'POR'): 80}

# Define the variable for the flow on each arc, bounded by the arc capacities
# upper and lower bound 
model.flow = Var(arc_capacities.keys(), domain=NonNegativeReals, bounds= lambda model, i, j: (0, arc_capacities[i, j]))

# Objective: Maximize the flow into LA
model.objective = Objective(expr=sum(model.flow[i, j] for (i, j) in arc_capacities.keys() if j == 'POR'), sense=maximize)

# Flow balance constraints: For each node except POR and DET, the inflow must equal the outflow
def flow_balance_rule(model, node):
    inflow = sum(model.flow[i, j] for (i, j) in arc_capacities.keys() if j == node)
    outflow = sum(model.flow[i, j] for (i, j) in arc_capacities.keys() if i == node)
    if node == 'DET':
        return outflow == sum(model.flow[node, j] for j in nodes if (node, j) in arc_capacities.keys())  # Source node: only outflow
    elif node == 'POR':
        return inflow == sum(model.flow[i, node] for i in nodes if (i, node) in arc_capacities.keys())  # Sink node: only inflow
    elif node == 'MIN':
        
        return outflow = inflow-20 
    else:
        return inflow == outflow  # Intermediate nodes: inflow must equal outflow

model.flow_balance = Constraint(nodes, rule=flow_balance_rule)

# Display the model
model.pprint()